## Embedding Model Artefacts

The notebook shows how to produce a model and artefacts for a model that generates predictions of embedding components and the result is computed as the minimum distance to elements of an embedding dataset with a configurable distance function. The artefacts produced are:
* tensorflow saved model
* embedding dataset
* verification samples

In [ ]:
import json
import tensorflow as tf
import tensorflow.keras as K
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np

### Example Word2Vec Model Construction

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

In [ ]:
encoder = info.features['text'].encoder
embedding_dim=16

model = K.Sequential([
  K.layers.Embedding(encoder.vocab_size, embedding_dim),
  K.layers.GlobalAveragePooling1D(),
  K.layers.Dense(16, activation='relu'),
  K.layers.Dense(1)
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

#### Make a model that predicts the embedding components

In [ ]:
prediction_model = K.models.clone_model(model)
prediction_model.pop()
prediction_model.pop()
prediction_model.summary()

#### Extract embedding weights from the model

In [ ]:
embedding_weights = model.layers[0].get_weights()[0][1:,:]
embedding_weights.shape

#### Generate verification data from model and embeddings

In this example we compute the cosine distance for verification.

In [ ]:
verification_input = test_batches.unbatch().batch(1).take(10)
num_results = 5
requests = [{
    'input': [[int(x) for x in e[0][0]]],
    'num': num_results,
    'distance': 'cosine'
} for e in list(verification_input.as_numpy_iterator())]

prediction_output = prediction_model.predict(verification_input)

def norm(m):
    return m / np.sqrt(np.sum(m * m, axis=-1, keepdims=1))

scores = norm(prediction_output) @ norm(embedding_weights).T

examples = prediction_output.shape[0]
scored_ix = np.arange(examples).reshape(-1, 1)
top_k = scores.argpartition(-num_results)[:,-num_results:]
sorted_k = top_k[scored_ix, (scores[scored_ix, top_k]).argsort()]
scores_k = scores[scored_ix, sorted_k]

responses = [[{'term': encoder.decode([i + 1]).rstrip(), 'score': float(s)} for i, s in zip(terms, scores)] for terms, scores in zip(top_k, scores_k)]

#### Write out all Artefacts

In [ ]:
!mkdir -p /tmp/word2vec/model
prediction_model.save('/tmp/word2vec/model')

!tar -cvzf /tmp/word2vec/model.tgz -C /tmp/word2vec/model .
!rm -fr /tmp/word2vec/model

pd.DataFrame(
    embedding_weights,
    index=pd.Index(
        [encoder.decode([i]).rstrip() for i in range(1, encoder.vocab_size)],
        name='term')
).to_csv('/tmp/word2vec/embedding.csv')

with open('/tmp/word2vec/verification.jsonl', 'wt') as f:
    for req, resp in zip(requests, responses):
        json.dump({'request': req, 'response': resp}, f)
        f.write('\n')

!ls -l /tmp/word2vec